- 데이터 소스: 국토교통부 실거래가 공개시스템: https://rtmobile.molit.go.kr/pt/xls/xls.do?mobileAt=

In [2]:
import pandas as pd          # csv 파일 read
import statsmodels.api as sm # 회귀분석 모델 생성
import joblib                # pkl 모델 저장하거나 load

In [5]:
df = pd.read_csv('../data/trade_apt_api.csv', encoding = 'cp949')
# 결측치 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   거래금액     318 non-null    int64  
 1   건축년도     318 non-null    int64  
 2   년        318 non-null    int64  
 3   법정동      318 non-null    object 
 4   아파트      318 non-null    object 
 5   월        318 non-null    int64  
 6   일        318 non-null    int64  
 7   전용면적     318 non-null    float64
 8   지번       318 non-null    object 
 9   지역코드     318 non-null    int64  
 10  층        318 non-null    int64  
 11  해제사유발생일  318 non-null    object 
 12  해제여부     318 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 32.4+ KB


In [6]:
df.head(1)

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,80000,2002,2021,신교동,신현(101동),8,16,84.82,6-13,11110,1,-,-


In [10]:
# 독립변수
# 분석의 편의상 모델 단순화를 위해 법정동(지역)은 독립변수에서 제외
# 법정동명은 One-Hot Encoding 필요하기 때문
x = df[['건축년도', '전용면적', '층']]
# 상수항(1) 추가하기 위한 add_constant 함수
X = sm.add_constant(x)
# 종속변수
y = df['거래금액']
print('독립변수 X와 타겟변수 y의 shape:', X.shape, y.shape)

독립변수 X와 타겟변수 y의 shape: (318, 4) (318,)


In [12]:
model = sm.OLS(y, X).fit()
model.summary()
# R-squared(r²): 0.648, 독립변수가 종속변수를 64.8% 설명한다는 것을 의미
# Adj.R-squared(Adjust R-squared, 수정된 r²): 64.4% 설명

# F-statistic: 1을 기준으로 1보다 크면 분석결과 유의미
# Prob(확률) (F-statistic): 8.54e-71, F-통계량의 확률이 0.05보다 작으면 유의미

# P>|t|: p-value가 0.5보다 낮아야 분석결과가 유의미
# coef: 상관관계 +면 양의 상관관계, -면 음의 상관관계
# std err: 표준 오차

# Durbin-Watson: 1.352: 2에 가까울수록 상관없음

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   거래금액   R-squared:                       0.648
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     192.4
Date:                Mon, 06 Jan 2025   Prob (F-statistic):           8.54e-71
Time:                        10:32:42   Log-Likelihood:                -3777.5
No. Observations:                 318   AIC:                             7563.
Df Residuals:                     314   BIC:                             7578.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.855e+06   4.25e+05     -9.069      0.000   -4.69e+06   -3.02e+06
건축년도        1925.6916    212.616      9.057      0.000    1507.360    2344.023
전용면적         962.1507     47.367     20.313      0.000     868.955    1055.347
층           2058.1524    417.716      4.927      0.000    1236.276    2880.028
==============================================================================
Omnibus:                       20.985   Durbin-Watson:                   1.352
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               42.734
Skew:                           0.345   Prob(JB):                     5.25e-10
Kurtosis:                       4.658   Cond. No.                     4.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
# 거래금액(y)의 단위는 만 원
# 80000 → 8억
X.iloc[0], y[0]

(const       1.00
 건축년도     2002.00
 전용면적       84.82
 층           1.00
 Name: 0, dtype: float64,
 80000)

In [14]:
model.predict([[1, 2002, 84.32, 1]])

array([83308.06947848])

In [16]:
# Model 저장
joblib.dump(model, '../model/ex1_apt_price_regression.pkl')

['../model/ex1_apt_price_regression.pkl']

In [24]:
# '건축년도', '전용면적', '층'을 입력받아 아파트 가격 예측하는 함수 생성
def predict_apt_price(year, square, floor):
    loaded_model = joblib.load('../model/ex1_apt_price_regression.pkl')
    input_data = [[1, year, square, floor]]
    results = loaded_model.predict(input_data)
    result = round(results[0]*10000) # 단위 만 원 → 원을 변경 / 반올림
    return format(result, ',') + '원입니다'

In [25]:
year   = int(input('몇 년에 건축되었나요? > '))
square = int(input('평수는 몇 제곱미터인가요? > '))
floor  = int(input('아파트는 몇 층인가요? > '))
predict_apt_price(year, square, floor)

몇 년에 건축되었나요? > 2001
평수는 몇 제곱미터인가요? > 105
아파트는 몇 층인가요? > 22


'1,445,008,561원입니다'